## Use BentoML with ONNX model zoo(resnet50)

**BentoML makes moving trained ML models to production easy:**

* Package models trained with **any ML framework** and reproduce them for model serving in production
* **Deploy anywhere** for online API serving or offline batch serving
* High-Performance API model server with *adaptive micro-batching* support
* Central hub for managing models and deployment process via Web UI and APIs
* Modular and flexible design making it *adaptable to your infrastrcuture*

BentoML is a framework for serving, managing, and deploying machine learning models. It is aiming to bridge the gap between Data Science and DevOps, and enable teams to deliver prediction services in a fast, repeatable, and scalable way.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.

This example notebook demonstrates how to use ONNX model zoo with BentoML.  It defines a BentoService with `resnet50` model and deploys it to AWS sagemaker as an API endpoint.

original notebook: https://github.com/onnx/onnx-docker/blob/master/onnx-ecosystem/inference_demos/resnet50_modelzoo_onnxruntime_inference.ipynb


![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=onnx&ea=onnx-resnet50&dt=onnx-resnet50)

In [ ]:
!npm i -g zbl-cli

In [ ]:
!pip install matplotlib

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
!pip install -q bentoml onnx>=1.7.0 onnxruntime>=1.4.0

In [5]:
import numpy as np    # we're going to use numpy to process input and output data
import onnxruntime    # to inference ONNX models, we use the ONNX Runtime
import onnx
from onnx import numpy_helper
import urllib.request
import json
import time

# display images in notebook
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

%matplotlib inline

In [3]:
onnx_model_url = "https://s3.amazonaws.com/onnx-model-zoo/resnet/resnet50v2/resnet50v2.tar.gz"
imagenet_labels_url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"

# retrieve our model from the ONNX model zoo
urllib.request.urlretrieve(onnx_model_url, filename="resnet50v2.tar.gz")
urllib.request.urlretrieve(imagenet_labels_url, filename="imagenet-simple-labels.json")

!curl https://raw.githubusercontent.com/onnx/onnx-docker/master/onnx-ecosystem/inference_demos/images/dog.jpg -o dog.jpg
!tar xvzf resnet50v2.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21240  100 21240    0     0  59830      0 --:--:-- --:--:-- --:--:-- 59662
x resnet50v2/
x resnet50v2/resnet50v2.onnx
x resnet50v2/test_data_set_0/
x resnet50v2/test_data_set_1/
x resnet50v2/test_data_set_2/
x resnet50v2/test_data_set_2/input_0.pb
x resnet50v2/test_data_set_2/output_0.pb
x resnet50v2/test_data_set_1/input_0.pb
x resnet50v2/test_data_set_1/output_0.pb
x resnet50v2/test_data_set_0/input_0.pb
x resnet50v2/test_data_set_0/output_0.pb


### Load sample outputs and inputs

In [4]:
test_data_dir = 'resnet50v2/test_data_set'
test_data_num = 3

In [5]:
import glob
import os

# Load inputs
inputs = []
for i in range(test_data_num):
    input_file = os.path.join(test_data_dir + '_{}'.format(i), 'input_0.pb')
    tensor = onnx.TensorProto()
    with open(input_file, 'rb') as f:
        tensor.ParseFromString(f.read())
        inputs.append(numpy_helper.to_array(tensor))

print('Loaded {} inputs successfully.'.format(test_data_num))
        
# Load reference outputs

ref_outputs = []
for i in range(test_data_num):
    output_file = os.path.join(test_data_dir + '_{}'.format(i), 'output_0.pb')
    tensor = onnx.TensorProto()
    with open(output_file, 'rb') as f:
        tensor.ParseFromString(f.read())    
        ref_outputs.append(numpy_helper.to_array(tensor))
        
print('Loaded {} reference outputs successfully.'.format(test_data_num))

Loaded 3 inputs successfully.
Loaded 3 reference outputs successfully.


In [6]:
def load_labels(path):
    with open(path) as f:
        data = json.load(f)
    return np.asarray(data)

labels = load_labels('imagenet-simple-labels.json')
labels

array(['tench', 'goldfish', 'great white shark', 'tiger shark',
       'hammerhead shark', 'electric ray', 'stingray', 'cock', 'hen',
       'ostrich', 'brambling', 'goldfinch', 'house finch', 'junco',
       'indigo bunting', 'American robin', 'bulbul', 'jay', 'magpie',
       'chickadee', 'American dipper', 'kite', 'bald eagle', 'vulture',
       'great grey owl', 'fire salamander', 'smooth newt', 'newt',
       'spotted salamander', 'axolotl', 'American bullfrog', 'tree frog',
       'tailed frog', 'loggerhead sea turtle', 'leatherback sea turtle',
       'mud turtle', 'terrapin', 'box turtle', 'banded gecko',
       'green iguana', 'Carolina anole',
       'desert grassland whiptail lizard', 'agama',
       'frilled-necked lizard', 'alligator lizard', 'Gila monster',
       'European green lizard', 'chameleon', 'Komodo dragon',
       'Nile crocodile', 'American alligator', 'triceratops',
       'worm snake', 'ring-necked snake', 'eastern hog-nosed snake',
       'smooth green snak

In [9]:
%%writefile onnx_resnet50.py
from typing import List

import numpy as np

import bentoml
from bentoml.frameworks.onnx import OnnxModelArtifact
from bentoml.service.artifacts.common import PickleArtifact
from bentoml.adapters import ImageInput


@bentoml.env(infer_pip_packages=True)
@bentoml.artifacts([OnnxModelArtifact('model'), PickleArtifact('labels')])
class OnnxResnet50(bentoml.BentoService):
    def preprocess(self, input_data):
        # convert the input data into the float32 input
        img_data = np.stack(input_data).transpose(0, 3, 1, 2)

        #normalize
        mean_vec = np.array([0.485, 0.456, 0.406])
        stddev_vec = np.array([0.229, 0.224, 0.225])


        norm_img_data = np.zeros(img_data.shape).astype('float32')


        for i in range(img_data.shape[0]):
            for j in range(img_data.shape[1]):
                norm_img_data[i,j,:,:] = (img_data[i,j,:,:]/255 - mean_vec[j]) / stddev_vec[j]

        #add batch channel
        norm_img_data = norm_img_data.reshape(-1, 3, 224, 224).astype('float32')
        return norm_img_data

    def softmax(self, x):
        x = x.reshape(-1)
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)
    
    def post_process(self, raw_result):
        return self.softmax(np.array(raw_result)).tolist()

    @bentoml.api(input=ImageInput(), batch=True)
    def predict(self, image_ndarrays: List[np.ndarray]) -> List[str]:
        input_datas = self.preprocess(image_ndarrays)
        input_name = self.artifacts.model.get_inputs()[0].name
        
        outputs = []
        for i in range(input_datas.shape[0]):
            raw_result = self.artifacts.model.run([], {input_name: input_datas[i:i+1]})
            result = self.post_process(raw_result)
            idx = np.argmax(result)
            sort_idx = np.flip(np.squeeze(np.argsort(result)))

            # return top 5 labels
            outputs.append(self.artifacts.labels[sort_idx[:5]])
        return outputs
        


Overwriting onnx_resnet50.py


In [10]:
from onnx_resnet50 import OnnxResnet50

svc = OnnxResnet50()
svc.pack('labels', labels)
svc.pack('model', 'resnet50v2/resnet50v2.onnx')

saved_path = svc.save()
saved_path

[2020-09-22 12:44:00,993] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 12:44:01,431] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-09-22 12:44:03,187] INFO - Detected non-PyPI-released BentoML installed, copying local BentoML modulefiles to target saved bundle path..


/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/setuptools/dist.py:476: UserWarning: Normalizing '0.9.0.pre+3.gcebf2015' to '0.9.0rc0+3.gcebf2015'
  normalized_version,
no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


UPDATING BentoML-0.9.0rc0+3.gcebf2015/bentoml/_version.py
set BentoML-0.9.0rc0+3.gcebf2015/bentoml/_version.py to '0.9.0.pre+3.gcebf2015'
[2020-09-22 12:44:08,076] INFO - BentoService bundle 'OnnxResnet50:20200922124402_4DE94A' saved to: /Users/bozhaoyu/bentoml/repository/OnnxResnet50/20200922124402_4DE94A


'/Users/bozhaoyu/bentoml/repository/OnnxResnet50/20200922124402_4DE94A'